<a href="https://colab.research.google.com/github/lhagen-isb/Community-Notebooks/blob/master/Beta/Exploring_TCGA_Clinical_Data_and_Creating_Cohorts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating TCGA cohorts

This notebook will show you how to create a TCGA cohort using the publicly available TCGA BigQuery tables that the [ISB-CGC](http://isb-cgc.org) project has produced based on the open-access [TCGA](http://cancergenome.nih.gov/) data available at the [Data Portal](https://tcga-data.nci.nih.gov/tcga/).  You will need to have access to a Google Cloud Platform (GCP) project in order to use BigQuery.  If you don't already have one, you can sign up for a [free-trial](https://cloud.google.com/free-trial/).

We are not attempting to provide a thorough BigQuery or IPython tutorial here, as a wealth of such information already exists.  Here are some links to some resources that you might find useful: 
* [BigQuery](https://cloud.google.com/bigquery/what-is-bigquery)
* the BigQuery [web UI](https://console.cloud.google.com/bigquery) 
    * where you can run queries interactively
* [Jupyter Notebooks](http://jupyter.org/)
* [Google Cloud Datalab](https://cloud.google.com/datalab/) 
    * interactive cloud-based platform for analyzing data built on the Jupyter Notebooks
* [Google Colaboratory](https://colab.research.google.com/)
    * Free Jupyter Notebook environment that runs in your browser

There are also many tutorials and samples available on github (see, in particular, the [datalab](https://github.com/GoogleCloudPlatform/datalab) repo, the [Google Genomics](  https://github.com/googlegenomics) project), and our own [Community Notebooks](https://github.com/isb-cgc/Community-Notebooks).

OK then, let's get started!  In order to work with BigQuery, the first thing you need to do is import the bigquery module:

In [0]:
from google.cloud import bigquery

Next we will need to Authorize our access to BigQuery and the Google Cloud. For more information see ['Quick Start Guide to ISB-CGC'](INSERT LINK) and alternative authentication methods can be found [here](https://googleapis.github.io/google-cloud-python/latest/core/auth.html).

In [0]:
#@title Default title text
!gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&access_type=offline


Enter verification code: 4/ZwE1UWpR6CYqyXz4JZWW9oGQ3nV2_wehPckfS_YA7vzCERAyp9Z9gW4

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests
Application Default Credentials.

To generate an access token for other uses, run:
  gcloud auth application-default print-access-token


To take a quick anonymous survey, run:
  $ gcloud alpha survey



In [0]:
# Create a variable for which client to use with BigQuery
client = bigquery.Client('PROJECT ID') # Replace PROJECT ID with your project ID
# Create a reference for the data set you are going to be using
dataset_ref = client.dataset('TCGA_bioclin_v0', project='PROJECT ID') #replace PROJECT ID with your project ID

The next thing you need to know is how to access the specific tables you are interested in.  BigQuery tables are organized into datasets, and datasets are owned by a specific GCP project.  The tables we will be working with in this notebook are in a dataset called **`TCGA_bioclin_v0`**, owned by the **`isb-cgc`** project.  A full table identifier is of the form `<project_id>.<dataset_id>.<table_id>`.  Let's start by getting some basic information about the tables in this dataset:

In [0]:
# For each table in the dataset print the number of rows,
# number of bytes and the name of the table
for t in client.list_tables(dataset_ref):
  table_ref = dataset_ref.table(t.table_id)
  table = client.get_table(table_ref)
  print '%10d rows  %12d bytes   %s' \
      % (table.num_rows, table.num_bytes, t.table_id)

      6322 rows       1725830 bytes   Annotations
     23797 rows       7308232 bytes   Biospecimen
     11315 rows       4801321 bytes   Clinical


In this tutorial, we are going to look at a few different ways that we can use the information in these tables to create cohorts.  Now, you maybe asking what we mean by "cohort" and why you might be interested in *creating* one, or maybe what it even means to "create" a cohort.  The TCGA dataset includes clinical, biospecimen, and molecular data from over 10,000 cancer patients who agreed to be a part of this landmark research project to build [The Cancer Genome Atlas](http://cancergenome.nih.gov/).  This large dataset was originally organized and studied according to [cancer type](http://cancergenome.nih.gov/cancersselected) but now that this multi-year project is nearing completion, with over 30 types of cancer and over 10,000 tumors analyzed, **you** have the opportunity to look at this dataset from whichever angle most interests you.  Maybe you are particularly interested in early-onset cancers, or gastro-intestinal cancers, or a specific type of genetic mutation.  This is where the idea of a "cohort" comes in.  The original TCGA "cohorts" were based on cancer type (aka "study"), but now you can define a cohort based on virtually any clinical or molecular feature by querying these BigQuery tables.  A cohort is simply a list of samples, using the [TCGA barcode](https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/) system.  Once you have created a cohort you can use it in any number of ways: you could further explore the data available for one cohort, or compare one cohort to another, for example.

In the rest of this tutorial, we will create several different cohorts based on different motivating research questions.  We hope that these examples will provide you with a starting point from which you can build, to answer your own research questions.

## Exploring the Clinical data table
Let's start by looking at the clinical data table.  The TCGA dataset contains a few very basic clinical data elements for almost all patients, and contains additional information for some tumor types only.  For example smoking history information is generally available only for lung cancer patients, and BMI (body mass index) is only available for tumor types where that is a known significant risk factor.  Let's take a look at the clinical data table and see how many different pieces of information are available to us:

#### Get Table Schema

In [0]:
# Magic command of bigquery with the project id as PROJECT_ID and create a Pandas Dataframe
%%bigquery --project PROJECT ID
# Then query the table for the column names
SELECT column_name
FROM `isb-cgc`.TCGA_bioclin_v0.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'Clinical'

# Syntax of the above query
# SELECT * 
# FROM `project_name.dataset_name.INFORMATION_SCHEMA.COLUMNS`
# WHERE table_catalog=project_name and table_schema=dataset_name and table_name=table_name

,column_name
0,program_name
1,case_barcode
2,case_gdc_id
3,program_dbgap_accession_number
4,project_short_name


That's a lot of fields!  We can also get at the schema programmatically:

#### Programmatically Get Schema

In [0]:
# Create a reference for the table
table_ref = dataset_ref.table('Clinical')

# Get the table
table = client.get_table(table_ref)
# Create a list of the field names
fieldNames = list(map(lambda tsf: tsf.name, table.schema))
#Create a list of the field types
fieldTypes = list(map(lambda tsf: tsf.field_type, table.schema))
# Print the number of fields and the first 5 fields
print " This table has %d fields. " % ( len(fieldNames) )
print " The first few field names and types are: "
for i in range(5):
  print fieldNames[i], fieldTypes[i]

 This table has 73 fields. 
 The first few field names and types are: 
program_name STRING
case_barcode STRING
case_gdc_id STRING
program_dbgap_accession_number STRING
project_short_name STRING


Let's look at these fields and see which ones might be the most "interesting", by looking at how many times they are filled-in (not NULL), or how much variation exists in the values.  If we wanted to look at just a single field, "tobacco_smoking_history" for example, we could use a very simple query to get a basic summary:

In [0]:
%%bigquery --project PROJECT ID
SELECT tobacco_smoking_history,
COUNT(*) AS n
FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
GROUP BY tobacco_smoking_history
ORDER BY n DESC

,tobacco_smoking_history,n
0,NaN,8316
1,1.0,865
2,4.0,799
3,2.0,710
4,3.0,568
5,5.0,57


But if we want to loop over *all* fields and get a sense of which fields might provide us with useful criteria for specifying a cohort, we'll want to automate that.  We'll put a threshold on the minimum number of patients that we expect information for, and the maximum number of unique values (since fields such as the "ParticipantBarcode" will be unique for every patient and, although we will need that field later, it's probably not useful for defining a cohort).

#### Find Interesting Fields

In [0]:
# Get the number of rows
numPatients = table.num_rows
# Print the total number of patients
print(" The %s table describes a total of %d patients. " % ( table.table_id, numPatients ))

# let's set a threshold for the minimum number of values that a field should have,
# the maximum number of unique values, and either the highest cancer type or
# the mean and sigma of the row.
minNumPatients = int(numPatients*0.80)
maxNumValues = 50

# Create a variable to be filled in by the for loop with the number
# interesting features
numInteresting = 0
# Create a list to hold the results from the loop below
iList = []

# Loop over the fields and find the number of values with the number of unique
# values and the 
for iField in range(len(fieldNames)):
  aField = fieldNames[iField]
  aType = fieldTypes[iField]
  qString = "SELECT {0} FROM `isb-cgc.TCGA_bioclin_v0.Clinical`".format(aField)
  df = client.query(qString).result().to_dataframe()
  summary = df[str(aField)].describe()
  if ( aType == "STRING" ):
    topFrac = float(summary['freq'])/float(summary['count'])
    if ( summary['count'] >= minNumPatients ):
      if ( summary['unique'] <= maxNumValues and summary['unique'] > 1 ):
        if ( topFrac < 0.90 ):
          numInteresting += 1
          iList += [aField]
          print("     > %s has %d values with %d unique (%s occurs %d times) " % (str(aField), summary['count'], summary['unique'], summary['top'], summary['freq']))
  else:
    if ( summary['count'] >= minNumPatients ):
      if ( summary['std'] > 0.1 ):
        numInteresting += 1
        iList += [aField]
        print("     > %s has %d values (mean=%.0f, sigma=%.0f) " % (str(aField), summary['count'], summary['mean'], summary['std']))

print(" ")
print(" Found %d potentially interesting features: " % numInteresting)
print("   ", iList)

 The Clinical table describes a total of 11315 patients. 
     > project_short_name has 11315 values with 33 unique (TCGA-BRCA occurs 1098 times) 
     > project_name has 11315 values with 33 unique (Breast Invasive Carcinoma occurs 1098 times) 
     > disease_code has 11315 values with 33 unique (BRCA occurs 1098 times) 
     > gender has 11160 values with 2 unique (FEMALE occurs 5815 times) 
     > vital_status has 11156 values with 2 unique (Alive occurs 7534 times) 
     > race has 9835 values with 5 unique (WHITE occurs 8186 times) 
     > age_at_diagnosis has 11109 values (mean=59, sigma=14) 
     > days_to_birth has 11041 values (mean=-21763, sigma=5266) 
     > days_to_last_known_alive has 11102 values (mean=1037, sigma=1041) 
     > year_of_initial_pathologic_diagnosis has 11030 values (mean=2008, sigma=4) 
     > person_neoplasm_cancer_status has 10236 values with 2 unique (TUMOR FREE occurs 6507 times) 
     > batch_number has 11160 values (mean=203, sigma=135) 
 
 Found 12 

The above helps us narrow down on which fields are likely to be the most useful, but if you have a specific interest, for example in menopause or HPV status, you can still look at those in more detail very easily: 

In [0]:
%%bigquery --project PROJECT ID 
SELECT menopause_status, COUNT(*) AS n
FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE menopause_status IS NOT NULL
GROUP BY menopause_status
ORDER BY n DESC

,menopause_status,n
0,Post (prior bilateral ovariectomy OR >12 mo si...,1291
1,Pre (<6 months since LMP AND no prior bilatera...,389
2,Peri (6-12 months since last menstrual period),82
3,Indeterminate (neither Pre or Postmenopausal),54


We might wonder which specific tumor types have menopause information:

In [0]:
%%bigquery --project PROJECT ID 
SELECT project_short_name, COUNT(*) AS n
FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE menopause_status IS NOT NULL
GROUP BY project_short_name
ORDER BY n DESC

,project_short_name,n
0,TCGA-BRCA,1007
1,TCGA-UCEC,517
2,TCGA-CESC,237
3,TCGA-UCS,55


In [0]:
%%bigquery --project PROJECT ID 
SELECT hpv_status, hpv_calls, COUNT(*) AS n
FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE hpv_status IS NOT NULL
GROUP BY hpv_status, hpv_calls
HAVING n > 20
ORDER BY n DESC

,hpv_status,hpv_calls,n
0,Negative,None,664
1,Positive,HPV16,238
2,Positive,HPV18,41
3,Positive,HPV33,25
4,Positive,HPV45,24


## TCGA Annotations

An additional factor to consider, when creating a cohort is that there may be additional information that might lead one to exclude a particular patient from a cohort.  In certain instances, patients have been redacted or excluded from analyses for reasons such as prior treatment, etc, but since different researchers may have different criteria for using or excluding certain patients or certain samples from their analyses, an overview of the annoations can be found [here](https://docs.gdc.cancer.gov/Encyclopedia/pages/Annotations_TCGA/).  These annotations have also been uploaded into a BigQuery table and can be used in conjuction with the other BigQuery tables.

# Create a Cohort from Two Tables

Now that we have a better idea of what types of information is available in the Clinical data table, let's create a cohort consisting of female breast-cancer patients, diagnosed at the age of 50 or younger.

In this next code cell, we define several queries with a **`WITH`** clause which allows us to use them in a final query. We will then save the query to a [Pandas DataFrame](https://pandas.pydata.org/) to allow it to be analyzed later with a named data frame.
* the first query, called **`select_on_annotations`**, finds all patients in the Annotations table which have either been 'redacted' or had 'unacceptable prior treatment';  
* the second query, **`select_on_clinical`** selects all female breast-cancer patients who were diagnosed at age 50 or younger, while also pulling out a few additional fields that might be of interest;  and
* the final query joins these two together and returns just those patients that meet the clinical-criteria and do **not** meet the exclusion-criteria.

#### Create a Query for a Cohort from Two Tables

In [7]:
# First use the BigQuery Magic Command (%%bigquery) then name the dataframe
# (early_onset_breast_canver), and finallly include your project ID
# (--project PROJECT ID).
%%bigquery early_onset_breast_cancer --project PROJECT ID
WITH
  select_on_annotations AS (
  SELECT
    case_barcode,
    category AS categoryName,
    classification AS classificationName
  FROM
    `isb-cgc.TCGA_bioclin_v0.Annotations`
  WHERE
    ( entity_type="Patient"
      AND (category="History of unacceptable prior treatment related to a prior/other malignancy"
        OR classification="Redaction" ) )
  GROUP BY
    case_barcode,
    categoryName,
    classificationName
),
--
select_on_clinical AS (
  SELECT
    case_barcode,
    vital_status,
    days_to_last_known_alive,
    ethnicity,
    histological_type,
    menopause_status,
    race
  FROM
    `isb-cgc.TCGA_bioclin_v0.Clinical`
  WHERE
    ( disease_code = "BRCA"
      AND age_at_diagnosis<=50
      AND gender="FEMALE" )
)
--
SELECT
  case_barcode
FROM (
  SELECT
    a.categoryName,
    a.classificationName,
    c.case_barcode
  FROM select_on_annotations AS a
  FULL JOIN select_on_clinical AS c
  ON
    a.case_barcode = c.case_barcode
  WHERE
    a.case_barcode IS NOT NULL
    OR c.case_barcode IS NOT NULL
  ORDER BY
    a.classificationName,
    a.categoryName,
    c.case_barcode
)
WHERE
  categoryName IS NULL
  AND classificationName IS NULL
  AND case_barcode IS NOT NULL
ORDER BY
  case_barcode

,case_barcode
0,TCGA-3C-AALI
1,TCGA-4H-AAAK
2,TCGA-5L-AAT0
3,TCGA-A1-A0SH
4,TCGA-A1-A0SJ
5,TCGA-A1-A0SN
6,TCGA-A1-A0SP
7,TCGA-A1-A0SQ
8,TCGA-A2-A04P
9,TCGA-A2-A04Q


### Useful Tricks
Before we leave off, here are a few useful tricks for working with BigQuery:
* If you want to see how much data and which tables are going to be touched by this data, you can use the "dry run" option.
* You can then build a query as a variable and put the results into a dataframe instead of 

In [0]:
# Create a variable with the query as a string
breast_cancer_query = """
WITH
  select_on_annotations AS (
  SELECT
    case_barcode,
    category AS categoryName,
    classification AS classificationName
  FROM
    `isb-cgc.TCGA_bioclin_v0.Annotations`
  WHERE
    ( entity_type="Patient"
      AND (category="History of unacceptable prior treatment related to a prior/other malignancy"
        OR classification="Redaction" ) )
  GROUP BY
    case_barcode,
    categoryName,
    classificationName
),
--
select_on_clinical AS (
  SELECT
    case_barcode,
    vital_status,
    days_to_last_known_alive,
    ethnicity,
    histological_type,
    menopause_status,
    race
  FROM
    `isb-cgc.TCGA_bioclin_v0.Clinical`
  WHERE
    ( disease_code = "BRCA"
      AND age_at_diagnosis<=50
      AND gender="FEMALE" )
)
--
SELECT
  case_barcode
FROM (
  SELECT
    a.categoryName,
    a.classificationName,
    c.case_barcode
  FROM select_on_annotations AS a
  FULL JOIN select_on_clinical AS c
  ON
    a.case_barcode = c.case_barcode
  WHERE
    a.case_barcode IS NOT NULL
    OR c.case_barcode IS NOT NULL
  ORDER BY
    a.classificationName,
    a.categoryName,
    c.case_barcode
)
WHERE
  categoryName IS NULL
  AND classificationName IS NULL
  AND case_barcode IS NOT NULL
ORDER BY
  case_barcode
"""

Since this is a large query, we might want to check the number of bytes that will be processed. We can use a dry run to see how many bytes wil be processed with the query without actually doing a query.

In [29]:
job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False
query_job = client.query(
    (breast_cancer_query),
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config,
)  # API request

# A dry run query completes immediately.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {} Kilobytes.".format(query_job.total_bytes_processed*0.001))

This query will process 788.043 Kilobytes.


For more information on price estimation, please see the [Estimating storage and query costs](https://cloud.google.com/bigquery/docs/estimate-costs) and [BigQuery best practices: Controlling costs](https://cloud.google.com/bigquery/docs/best-practices-costs) pages.

Below we can then use the same variable with the query to then run the query and put the result into a Pandas Dataframe for later analysis.

In [31]:
query = client.query(breast_cancer_query)
breast_cancer_query_results = query.result().to_dataframe()
breast_cancer_query_results.head(5)

,case_barcode
0,TCGA-3C-AALI
1,TCGA-4H-AAAK
2,TCGA-5L-AAT0
3,TCGA-A1-A0SH
4,TCGA-A1-A0SJ
